<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [7]</a>'.</span>

# Bottom age tracer in the OM3-25km beta release

In [1]:
#parameters

### USER EDIT start
esm_file='/g/data/ol01/access-om3-output/access-om3-025/MC_25km_jra_ryf-1.0-beta/experiment_datastore.json'
dpi=300
### USER EDIT stop

import os
from matplotlib import rcParams
%matplotlib inline
rcParams['figure.dpi']= dpi

plotfolder=f"/g/data/{os.environ['PROJECT']}/{os.environ['USER']}/access-om3-paper-figs/"
os.makedirs(plotfolder, exist_ok=True)

 # a similar cell under this means it's being run in batch
print("ESM datastore path: ",esm_file)
print("Plot folder path: ",plotfolder)

ESM datastore path:  /g/data/ol01/access-om3-output/access-om3-025/MC_25km_jra_ryf-1.0-beta/experiment_datastore.json
Plot folder path:  /g/data/tm70/cyb561/access-om3-paper-figs/


In [2]:
# Parameters
esm_file = "/g/data/ol01/access-om3-output/access-om3-025/25km-iaf-test-for-AK-expt-7df5ef4c/datastore.json"
plotfolder = "/g/data/tm70/cyb561/access-om3-paper-1/notebooks/mkfigs_output4/"


In [3]:
import intake
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cmocean as cm
from distributed import Client

# Stop annoying warnings coming out of xarray.
import warnings
warnings.filterwarnings('ignore')

In [4]:
client = Client(threads_per_worker=1) 
print(client.dashboard_link)

http://127.0.0.1:8787/status


Open intake datastore:

In [5]:
# datastore_path = "/g/data/ol01/access-om3-output/access-om3-025/MC_25km_jra_ryf-1.0-beta/experiment_datastore.json"

datastore = intake.open_esm_datastore(
    esm_file,
    columns_with_iterables=[
        "variable",
        "variable_long_name",
        "variable_standard_name",
        "variable_cell_methods",
        "variable_units"
    ]
)

6 Aug 2025: Output available from 1900-1948

### Load bottom age tracer

In [6]:
age_OM3 = datastore.search(variable="agessc", frequency="1mon").to_dask(
    xarray_open_kwargs = dict(
        chunks={"yh": -1, "xh": -1}, # Good for spatial operations, but not temporal
        # chunks={"time": -1}, # Good for temporal operations, but not spatial
        decode_timedelta=True
    )
)

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [7]:
# Year 10
age_OM3_10yr = age_OM3.agessc.sel(yh=slice(-90,-50)).sel(time=slice('1909-01-01','1909-12-31'))
age_OM3_10yr = age_OM3_10yr.mean('time').load()

KeyError: -90

In [ ]:
# Year 50
age_OM3_50yr = age_OM3.agessc.sel(yh=slice(-90,-50)).sel(time=slice('1950-01-01','1950-12-31')) # 1950 also avail for OM2-01
age_OM3_50yr = age_OM3_50yr.mean('time').load()

In [ ]:
# Get bottom tracer
depth_array = age_OM3_10yr * 0 + age_OM3_10yr.z_l
max_depth   = depth_array.max(dim='z_l', skipna=True)

In [ ]:
age_bot_10yr = age_OM3_10yr.where(depth_array.z_l >= max_depth)
age_bot_10yr = age_bot_10yr.sum(dim='z_l').load()

In [ ]:
age_bot_50yr = age_OM3_50yr.where(depth_array.z_l >= max_depth)
age_bot_50yr = age_bot_50yr.sum(dim='z_l').load()

Normalise age tracer by max value

In [ ]:
age_bot_10yr_normalised = age_bot_10yr/age_bot_10yr.max()
age_bot_50yr_normalised = age_bot_50yr/age_bot_50yr.max()

Mask contiental shelf

In [ ]:
# Load shelf mask: 1000 m isobath for OM3-25km grid, created using this notebook: Calc_OM3_25km_1000m_isobath_contour.ipynb (add to repository?)
contour_masked_above = xr.open_mfdataset('/g/data/x77/wgh581/Post_Process/DSW_in_025_models/Antarctic_slope_contour_1000m_OM3_25km.nc').contour_masked_above

shelf_mask = contour_masked_above.where(contour_masked_above==0, 1)
shelf_mask = shelf_mask.where(shelf_mask==1)

In [ ]:
age_bot_10yr_masked = age_bot_10yr_normalised*shelf_mask
age_bot_50yr_masked = age_bot_50yr_normalised*shelf_mask

### Plotting

In [ ]:
# Land mask
var_search = datastore.search(variable="deptho")
ds = var_search.search(path=var_search.df.path[0]).to_dask() # we only need one file as bathymetry doesn't change
bathymetry = ds["deptho"].load()
land = xr.where(np.isnan(bathymetry.rename("land")), 1, np.nan)
land = land.sel(yh=slice(-90,-50))

ft_size = 12 # fontsize
fig_path = '/g/data/x77/wgh581/Figures/DSW_in_025_models/'

In [ ]:
# 1000 m contour
slope_contour = contour_masked_above.where(contour_masked_above>0)
slope_contour = slope_contour/slope_contour

Plot OM3-25km at years 10 and 50

In [ ]:
plt.figure(figsize=(10, 6))

plt.subplot(211)
land.plot.contourf(colors="darkgrey", add_colorbar=False)
age_bot_10yr_masked.plot(cmap=cm.cm.rain_r, vmin=0.5, vmax=1, cbar_kwargs={'label': "Normalised bottom age"})
slope_contour.fillna(2).plot.contour(levels=[0,1], colors='darkgrey', linewidths=1.5)
plt.title('ACCESS-OM3-25km'), plt.title('a', fontweight='bold', fontsize=ft_size, loc='left')
plt.tick_params(labelbottom=False)
plt.xlabel(''), plt.ylabel('Latitude (°N)', fontsize=ft_size)
plt.text(-270, -79, 'Year 10', fontsize=ft_size)

plt.subplot(212)
land.plot.contourf(colors="darkgrey", add_colorbar=False)
age_bot_50yr_masked.plot(cmap=cm.cm.rain_r, vmin=0.5, vmax=1, cbar_kwargs={'label': "Normalised bottom age"})
slope_contour.fillna(2).plot.contour(levels=[0,1], colors='darkgrey', linewidths=1.5)
plt.title('b', fontweight='bold', fontsize=ft_size, loc='left')
plt.xlabel('Longitude (°E)', fontsize=ft_size), plt.ylabel('Latitude (°N)', fontsize=ft_size)
plt.text(-270, -79, 'Year 50', fontsize=ft_size)

# Save figure
save_flag = 1
if save_flag == 1:
    plt.savefig(fig_path + 'Fig_bottom_age_OM325km_yr10_yr50.png', dpi=300, bbox_inches='tight', facecolor='white');

Things to note:
- DSW overflow occurs mainly in the Ross Sea and also in the Weddell Sea
- No overflow in Adelie land (see pale yellow line that interrupts the young shelf waters from the young-ish offshore waters fed by the Ross Sea)
- Little overflow in Prydz Bay
- The young bottom waters offshore of the Antarctic continental shelf seem to be from DSW export. Deep open ocean convection - which often occurs in 0.25deg models - does not seem to impact Antarctic Bottom Water formation in OM3-25 km. This is great news! (Compare the mixed layer depth maps which show (i) very shallow maximum winter mixed layer depths and (ii) the pattern of young offshore age tracer does not align with where there are deeper mixed layers.)

### Compare to age tracer from OM2-01

Note there is no age tracer saved for OM2-025 (only for the first three years and then from 2300 onwards). OM2-01 has good DSW overflow and AABW formation so it makes sense to compare to that simulation.

In [ ]:
catalog  = intake.cat.access_nri
expt_01  = '01deg_jra55v13_ryf9091'
expt_025 = '025deg_jra55_ryf9091_gadi'

In [ ]:
age_OM2_01 = catalog[expt_01].search(variable="age_global", frequency="1mon").to_dask(
    xarray_open_kwargs = dict(
        chunks={"yh": -1, "xh": -1}, # Good for spatial operations, but not temporal
        # chunks={"time": -1}, # Good for temporal operations, but not spatial
        decode_timedelta=True
    )
)

In [ ]:
# Year 50
age_OM2_01_50yr = age_OM2_01.age_global.sel(yt_ocean=slice(-90,-50)).sel(time=slice('1950-01-01','1950-12-31'))
age_OM2_01_50yr = age_OM2_01_50yr.mean('time').load()

In [ ]:
# Get bottom tracer
depth_array_OM2_01 = age_OM2_01_50yr * 0 + age_OM2_01_50yr.st_ocean
max_depth_OM2_01   = depth_array_OM2_01.max(dim='st_ocean', skipna=True)

In [ ]:
age_bot_OM2_01_50yr = age_OM2_01_50yr.where(depth_array_OM2_01.st_ocean >= max_depth_OM2_01)
age_bot_OM2_01_50yr = age_bot_OM2_01_50yr.sum(dim='st_ocean').load()

In [ ]:
# Normalise age tracer
age_bot_OM2_01_50yr_normalised = age_bot_OM2_01_50yr/age_bot_OM2_01_50yr.max()

Mask continental shelf

In [ ]:
contour_data  = np.load('/g/data/g40/akm157/model_data/access-om2/Antarctic_slope_contour_1000m.npz')
contour_mask_numbered = contour_data['contour_mask_numbered']
contour_masked_above = contour_data['contour_masked_above']
contour_1000m = np.where(contour_mask_numbered == 0, contour_mask_numbered, 1000)
contour_xt_ocean = contour_data['xt_ocean']
contour_yt_ocean = contour_data['yt_ocean']

In [ ]:
# Create dataset
ds = xr.Dataset(
    {
        "contour_1000m": (("yt_ocean", "xt_ocean"), contour_1000m),
        "contour_mask_numbered": (("yt_ocean", "xt_ocean"), contour_mask_numbered),
        "contour_masked_above": (("yt_ocean", "xt_ocean"), contour_masked_above),
    },
    coords={
        "xt_ocean": contour_xt_ocean,
        "yt_ocean": contour_yt_ocean,
    },
)

In [ ]:
shelf_mask_01 = ds.contour_masked_above.where(ds.contour_masked_above==0, 1)
shelf_mask_01 = shelf_mask_01.where(shelf_mask_01==1)

In [ ]:
age_bot_OM2_01_50yr_masked = age_bot_OM2_01_50yr_normalised*shelf_mask_01

Plotting

In [ ]:
plt.figure(figsize=(10, 6))

plt.subplot(211)
land.plot.contourf(colors="darkgrey", add_colorbar=False)
age_bot_50yr_masked.plot(cmap=cm.cm.rain_r, vmin=0.5, vmax=1, cbar_kwargs={'label': "Normalised bottom age"})
slope_contour.fillna(2).plot.contour(levels=[0,1], colors='darkgrey', linewidths=1.5)
plt.title('ACCESS-OM3-25km'), plt.title('a', fontweight='bold', fontsize=ft_size, loc='left')
plt.tick_params(labelbottom=False)
plt.xlabel(''), plt.ylabel('Latitude (°N)', fontsize=ft_size)
plt.text(-270, -79, 'Year 50', fontsize=ft_size)

plt.subplot(212)
land.plot.contourf(colors="darkgrey", add_colorbar=False)
age_bot_OM2_01_50yr_masked.plot(cmap=cm.cm.rain_r, vmin=0.5, vmax=1, cbar_kwargs={'label': "Normalised bottom age"})
slope_contour.fillna(2).plot.contour(levels=[0,1], colors='darkgrey', linewidths=1.5)
plt.title('ACCESS-OM2-01'), plt.title('b', fontweight='bold', fontsize=ft_size, loc='left')
plt.xlabel('Longitude (°E)', fontsize=ft_size), plt.ylabel('Latitude (°N)', fontsize=ft_size)
plt.text(-270, -79, 'Year 50', fontsize=ft_size)

# Save figure
save_flag = 1
if save_flag == 1:
    plt.savefig(fig_path + 'Fig_bottom_age_OM325km_OM201_yr50.png', dpi=300, bbox_inches='tight', facecolor='white');